In [1]:
import numpy as np
import time

import naplib as nl
from auditory_cortex.datasets import BaselineDataset, DNNDataset
from auditory_cortex import utils

/home/ahmedb/projects/Wav2Letter/deepspeech.pytorch/deepspeech_pytorch/loader/data_loader.py:17: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("sox_io")


In [21]:
model_name = 'whisper_tiny'
session = 200206
bin_width = 50
layer_ID = 2
mVocs = True

dataset_obj = DNNDataset(
				session, bin_width, model_name, layer_ID, mVocs=mVocs
				)


tmax=300
tmin=0 
num_folds = 3

# function starts here...
tmin = tmin/1000
tmax = tmax/1000
sfreq = 1000/dataset_obj.bin_width
num_channels = dataset_obj.num_channels

# Deprecated...
mapping_set = dataset_obj.training_sent_ids

lmbdas = np.logspace(-2, 12, 15)
lmbda_score = np.zeros(((len(lmbdas), num_channels)))
np.random.shuffle(mapping_set)
size_of_chunk = int(len(mapping_set) / num_folds)

r = 0
print(f"\n For fold={r}: ")
if r<(num_folds-1):
	val_set = mapping_set[r*size_of_chunk:(r+1)*size_of_chunk]
else:
	val_set = mapping_set[r*size_of_chunk:]
train_set = mapping_set[np.isin(mapping_set, val_set, invert=True)]

train_x, train_y = dataset_obj.get_data(stim_ids=train_set)
val_x, val_y = dataset_obj.get_data(stim_ids=val_set)



# tempp
val_y = [val[:,:2] for val in val_y]
i = 0 
lmbda = lmbdas[i]


Using default normalizer file...
200206: creating DNNDataset for mVocs data..
NeuralData:  Creating object for session: 200206 ... Done.
DNNDataset: Loading data for session-200206 at bin_width-50ms.
DataLoader: Extracting spikes for session-200206...
NeuralData: Reading mVocs spikes for 200206. 
Reading raw features from /scratch/gilbreth/ahmedb/cache/mVocs/whisper_tiny/whisper_tiny_raw_features.pkl
Resamping ANN features at bin-width: 50

 For fold=0: 


#### starts from here...

In [126]:
class LinearModel:
	def __init__(self, alpha):
		"""Create linear model with regularization parameter alpha."""
		self.alpha = alpha

	def fit(self, X, y):
		X = cp.array(X)
		y = cp.array(y)
		self.Beta = utils.reg(X, y, lmbda=self.alpha)
		# self.model.fit(X, y)

	def predict(self, X):
		X = cp.array(X)
		pred = np.matmul(X, self.Beta)
		return cp.asnumpy(pred)
		
	
	def score(self, X, y):
		pred = self.predict(X)
		loss = -1*utils.mse_loss(y, pred)
		return loss
	
	@property
	def coef_(self):
		if not hasattr(self, 'Beta'):
			raise ValueError("Model has not been fit yet.")
		return cp.asnumpy(self.Beta)

In [127]:
from multiprocessing import Pool
from tqdm.auto import tqdm
import copy

class GpuTRF(nl.encoding.TRF):
	"""GPU accelerated TRF model."""
	def __init__(self, *args, **kwargs):
		super().__init__(*args, **kwargs)
		
		self.model = LinearModel(alpha=2)

	def fit(self, X, y):

		# gpu implementation
		# concatenate along 2nd last axis (the time axis),
		X = np.concatenate(X, axis=-2) 	
		y = np.concatenate(y, axis=0)

		if y.ndim == 1:
			y = y[:, np.newaxis]
		#
		self.ndim_y_ = y.ndim
		self.X_feats_ = X.shape[-1]
		self.n_targets_ = y.shape[1]
		self.n_models = None
		# self.y_feats_ = 1


		print(self.n_models)
		print(f"X shape: {X.shape}")
		print(f"y shape: {y.shape}")
		
		# Delay inputs and reshape
		if X.ndim == 3:
			# if there is layer axis in X, then we need to delay each layer separately
			self.n_models = X.shape[0]
			X_delayed = []
			for i in range(X.shape[0]):
				x_tmp, _ = self._delay_and_reshape(X[i])
				X_delayed.append(x_tmp)
			X_delayed = np.concatenate(X_delayed, axis=0)
		else:
			X_delayed, _ = self._delay_and_reshape(X)
		y_delayed = y
		
		print(f"X shape: {X_delayed.shape}")
		print(f"y shape: {y_delayed.shape}")

		self.model.fit(X_delayed, y_delayed)

		return self
	
	def predict(self, X):
		if not hasattr(self, 'X_feats_'):
			raise ValueError(f'Must call .fit() before can call .predict()')
		
		X = np.concatenate(X, axis=-2) 	

		# Delay inputs and reshape
		if X.ndim == 3:
			# if there is layer axis in X, then we need to delay each layer separately
			# self.n_models = X.shape[0]
			X_delayed = []
			for i in range(X.shape[0]):
				x_tmp, _ = self._delay_and_reshape(X[i])
				X_delayed.append(x_tmp)
			X_delayed = np.concatenate(X_delayed, axis=0)
		else:
			X_delayed, _ = self._delay_and_reshape(X)
		
		y_pred = self.model.predict(X_delayed)
		return y_pred
	
	def score(self, X, y):
		"""Compute the negative mean squared error of the model on the given data.
		"""
		X = np.concatenate(X, axis=-2) 	
		y = np.concatenate(y, axis=0)

		if y.ndim == 1:
			y = y[:, np.newaxis]

		# Delay inputs and reshape
		if X.ndim == 3:
			# if there is layer axis in X, then we need to delay each layer separately
			X_delayed = []
			for i in range(X.shape[0]):
				x_tmp, _ = self._delay_and_reshape(X[i])
				X_delayed.append(x_tmp)
			X_delayed = np.concatenate(X_delayed, axis=0)
		else:
			X_delayed, _ = self._delay_and_reshape(X)

		score = self.model.score(X_delayed, y)

		return score
	
	@property
	def coef_(self):
		if not hasattr(self, 'ndim_y_'):
			raise ValueError(f'Must call fit() first before accessing coef_ attribute.')
		if hasattr(self, 'n_models') and self.n_models is not None:
			return self.model.coef_.reshape(self.n_models, self.X_feats_, self._ndelays, self.n_targets_)
		else:
			return self.model.coef_.reshape(self.X_feats_, self._ndelays, self.n_targets_)
   

def _fit_channel(args):
	ch, X, y, model = args
	return ch, model.fit(X, y)

In [128]:
# from auditory_cortex.computational_models.encoding import GpuTRF
from sklearn.linear_model import RidgeCV, ElasticNet, Ridge, PoissonRegressor
estimator = Ridge(alpha=2)
trf_model = GpuTRF(
	tmin, tmax, sfreq, #estimator=estimator,
						n_jobs=1, show_progress=True
)
# trf_model = nl.encoding.TRF(
# 	tmin, tmax, sfreq, #estimator=estimator,
#                         n_jobs=1, show_progress=True
# )

In [135]:
train_x = [x[None,...] for x in train_x]
# train_x = [x.squeeze() for x in train_x]

In [136]:
print(f"x.shape: {train_x[0].shape}")
print(f"y.shape: {train_y[0].shape}")
out = trf_model.fit(X=train_x, y=train_y)

x.shape: (1, 13, 384)
y.shape: (13, 64)
None
X shape: (1, 9969, 384)
y shape: (9969, 64)
X shape: (9969, 2688)
y shape: (9969, 64)


In [137]:
trf_model.coef_.shape

(1, 384, 7, 64)

In [138]:
pred = trf_model.score(X=train_x, y=train_y)

In [140]:
pred.shape

(64,)

In [90]:
trf_model.model.Beta.shape

(2688, 64)

In [87]:
trf_model.model.Beta.shape

(2688, 64)

In [23]:
val_x = np.concatenate(val_x, axis=0)
val_y = np.concatenate(val_y, axis=0)

In [24]:
val_x.shape

(4968, 384)

In [25]:
x_tmp, y_tmp = trf_model._delay_and_reshape(val_x, val_y)

In [26]:
x_tmp.shape

(4968, 2688)

In [27]:
y_tmp.shape

(4968, 2)

In [ ]:
for xx, yy in zip(X, y):
			X_tmp, y_tmp = self._delay_and_reshape(xx, yy)

In [13]:
type(out)

__main__.GpuTRF

In [35]:
trf_model = nl.encoding.TRF(
						tmin, tmax, sfreq, #estimator=estimator,
						n_jobs=1, show_progress=True
						)

In [36]:
trf_model.tmax

0.3

In [37]:
xx = x[0]
yy = y[0]
x_tmp, y_tmp = trf_model._delay_and_reshape(xx, yy)

In [39]:
xx.shape

(30, 384)

In [40]:
x_tmp.shape

(30, 2688)

In [41]:
yy.shape

(30, 64)

In [42]:
y_tmp.shape

(30, 64)

In [43]:
import cupy as cp 
cp.cuda.Device(0).use()

<CUDA Device 0>

In [45]:
num_layers = 4
num_channels = 16
num_features = 100
num_samples = 500
X = cp.random.rand(num_layers, num_samples, num_features)
y = cp.random.rand(num_samples, num_channels)

In [49]:
X = []
for _ in range(10):
	X.append(cp.random.rand(2, num_samples, num_features))


X = np.concatenate(X, axis=-2)
print(X.shape)

(2, 5000, 100)


In [7]:
print(f"X.shape={X.shape}, y.shape={y.shape}")

X.shape=(4, 500, 100), y.shape=(500, 16)


In [16]:
B = utils.reg(X, y)

In [17]:
B.shape

(4, 100, 16)

In [19]:
X.shape

(4, 500, 100)

In [21]:
np.matmul(X, B).shape

(4, 500, 16)

In [18]:
cp.dot(X, B).shape

(4, 500, 4, 16)